# Agent V2: The Planner

A monolithic agent that plans work before executing it.
Extends the Coder with todo tools: file tools (9) + sandbox tools (1) + todo tools (6) = 16 tools.

In [ ]:
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.config.config import PROMPTS_DIR
from agentic_patterns.core.prompt import load_prompt
from agentic_patterns.tools.file import get_all_tools as get_file_tools
from agentic_patterns.tools.sandbox import get_all_tools as get_sandbox_tools
from agentic_patterns.tools.todo import get_all_tools as get_todo_tools

## Agent Definition

The system prompt instructs the agent to plan before executing. The agent receives
file tools, sandbox tools, and todo tools for task tracking.

In [ ]:
system_prompt = load_prompt(PROMPTS_DIR / "the_complete_agent" / "agent_planner.md")

tools = get_file_tools() + get_sandbox_tools() + get_todo_tools()
print(f"Total tools: {len(tools)}")
for t in tools:
    print(f"  {t.__name__}")

In [ ]:
agent = get_agent(system_prompt=system_prompt, tools=tools)

## Task: Multi-Step Data Processing

Ask the agent to perform a multi-step task. The agent should create a plan first,
then execute each step while tracking progress.

In [ ]:
prompt = """Create a CSV file with sample sales data (product, quantity, price) for 5 products.
Then write a Python script that reads the CSV, calculates the total revenue per product,
and prints a summary sorted by revenue. Save both files to /workspace/ and execute the script."""

agent_run, nodes = await run_agent(agent, prompt, verbose=True)

print("\n--- Agent Output ---")
print(agent_run.result.output)

## Verify

Check the files written to the workspace.

In [ ]:
from agentic_patterns.core.workspace import read_from_workspace

print("--- sales.csv ---")
print(read_from_workspace("/workspace/sales.csv"))
print("\n--- analyze_sales.py ---")
print(read_from_workspace("/workspace/analyze_sales.py"))

## Cleanup

In [ ]:
from agentic_patterns.core.sandbox.manager import SandboxManager
from agentic_patterns.core.user_session import get_user_id, get_session_id

SandboxManager().close_session(get_user_id(), get_session_id())